In [2]:
import nltk
from nltk.tokenize import sent_tokenize
from nltk import ngrams
from collections import Counter
import numpy as np

Sherlock Holmes corpus

In [3]:
file = open("./pg1661.txt",'r')
text = file.read()

In [4]:
sent_tokenize_list = sent_tokenize(text)

In [5]:
len(sent_tokenize_list)

6949

Q2: 80-20 train-test split

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(sent_tokenize_list,  test_size=0.20, random_state=7)

In [7]:
words_train =[]
words_test =[]
for i in range(len(X_train)):
    words_train +=[word.lower() for word in X_train[i].split() if word.isalpha()]
for i in range(len(X_test)):
    words_test +=[word.lower() for word in X_test[i].split() if word.isalpha()]

Q3: MLE_(n,test,train) takes n=1 for unigram , n=2 for bigram and so on. test  and train are the set of words on which we want to test and train.
The function returns two dictionaries each containing MLE values for each ngram for train and test set

In [8]:
def MLE_(n,train,test):
    
    mle_train = {}
    mle_test ={}
    if(n==1):
        n_gram_dict = Counter(train)
        for k in n_gram_dict.keys():
            mle_train[k] = n_gram_dict[k]/len(train)
        test_dict = Counter(words)
        for k in test_dict.keys():
            mle_test[k] = test_dict[k]/len(train)
        return mle_test, mle_train

    n_gram = list(ngrams(train, n))
    n_gram_dict = Counter(n_gram)
    if(n==2):
        n_1_gram = train
    else:
        n_1_gram = list(ngrams(train, n-1))
    n_1_gram_dict = Counter(n_1_gram)
    
    for k in n_gram_dict.keys():
        if(n==2):
            context = k[0]
        else:
            context = k[:-1]
        #print(context)
        mle_train[k] = n_gram_dict[k]/n_1_gram_dict[context]
 
      
    n_gram_test = list(ngrams(test, n))
    n_gram_dict_test = Counter(n_gram_test)
    
    
    for k in n_gram_dict_test.keys():
        if(n==2):
            context = k[0]
        else:
            context = k[:-1]
        #print(context)
        if(k in n_gram_dict.keys() and context in n_1_gram_dict.keys()):
            mle_test[k] = n_gram_dict[k]/n_1_gram_dict[context]
        else:
            mle_test[k] = 0
    return mle_test, mle_train
        
        
    

In [47]:
n =2#for bigram

In [48]:
mle_test, mle_train = MLE_(n,words_train,words_test)

In [49]:
mle_test

{('in', 'two'): 0,
 ('two', 'hours'): 0.01904761904761905,
 ('hours', 'we'): 0,
 ('we', 'must'): 0.02891566265060241,
 ('must', 'be'): 0.27007299270072993,
 ('be', 'on'): 0.002004008016032064,
 ('on', 'the'): 0.39100346020761245,
 ('the', 'scene'): 0.0008644910309055544,
 ('scene', 'of'): 0.6666666666666666,
 ('of', 'then'): 0.002215330084182543,
 ('then', 'suddenly'): 0.031007751937984496,
 ('suddenly', 'another'): 0,
 ('another', 'sound'): 0,
 ('sound', 'became'): 0,
 ('became', 'very'): 0,
 ('very', 'soothing'): 0,
 ('soothing', 'like'): 0,
 ('like', 'that'): 0.0547945205479452,
 ('that', 'of'): 0.01171875,
 ('of', 'a'): 0.06247230837394772,
 ('a', 'small'): 0.01675197766402978,
 ('small', 'jet'): 0,
 ('jet', 'of'): 0,
 ('of', 'steam'): 0,
 ('steam', 'escaping'): 0,
 ('escaping', 'continually'): 0,
 ('continually', 'from'): 0.5,
 ('from', 'a'): 0.06218905472636816,
 ('a', 'so'): 0.003257328990228013,
 ('so', 'say'): 0,
 ('say', 'the'): 0,
 ('the', 'to'): 0.0023773503349902745,
 ('to

In [12]:
len([k  for k in list(mle_test.values()) if k != 0] )

5073

for bigrams out of 11189, 5073 have non-zero probability, for trigrams out of 16139,  2752 have non-zero probabilty, and out of 17024 quadgrams, only 748 have non-zero probability out of 17135 5-grams, 204 have zon-zero probability

Q4(a). generate(n, word, sen_len) takes n=1 for unigram , n=2 for bigram and likewise, the starting word of the sentence, and length of sentence.The function returns a sentence of length sen_len 

In [13]:
def generate(n, word, sen_len):
    words = Counter(list(ngrams(words_train, n)))
    gram = sorted(words.items(), key=lambda x: -x[1])
    
    for i in range(sen_len):
        print (word,end=' ')
        
        choices = [element for element in gram if element[0][0] == word]
        
        if not choices:
            break
        total = sum(w for c, w in choices)
        proba = [y[1]/total for y in choices]
        
        c = np.random.multinomial(1, proba, size=1)

        word = choices[int(np.argmax(c,axis =1))][0][1]


In [14]:
lst = ['as','so','think','train','many']
for w in lst:
    generate(5,w,15)
    print("\n")


as sir managed that i shall spend in upon as she was that a hebrew 

so that you wish to one sorrow comes plays at having a perhaps i laughed 

think that francis hay i am my chair very vulnerable from and given him and 

train from grosvenor written a square is though he did not to them at the 

many in the united states copyright the strange the old man strikes even more this 



Q4(b). logProbability(sentence,n) calculates the log probability given sentence and specifying n= 1 for unigram , 2 for bigram and so on.

In [15]:
def logProbability(sentence,n):
    ngrams_train = Counter(list(ngrams(words_train, n)))
    if(n==2):
        n_1_grams_train = Counter(words_train)
    else:
        n_1_grams_train = Counter(list(ngrams(words_train, n-1)))
    ngrams_input  = list(ngrams(sentence.split(), n))
    prob = 0.0   
    #print(n_1_grams_train)
    for k in ngrams_input:
        if(n==2):
            context = k[0]
        else:
            context = k[:-1]
        
        if(k in ngrams_train.keys() and context in n_1_grams_train.keys()): 
            
            
            prob +=  -np.log((ngrams_train[k])/(n_1_grams_train[context]))

               
        else:
            
            prob += 0
    return prob

In [16]:
logProbability("it would be better for both of us",3)

4.828734701316095

Q5. smoothing(train,test) takes training and test set as input and calculates MLE for all the bigrams for both the set

In [17]:
def smoothing(train,test):
    words = train
    mle_train = {}
    mle_test ={}


    n_gram = list(ngrams(train, 2))
    n_gram_dict = Counter(n_gram)
    
    n_1_gram = words

    n_1_gram_dict = Counter(n_1_gram)
    
    for k in n_gram_dict.keys():
        context = k[0]

        mle_train[k] = n_gram_dict[k]/n_1_gram_dict[context]
 
      
    n_gram_test = list(ngrams(test, 2))
    n_gram_dict_test = Counter(n_gram_test)
    #print(n_gram_test)
    
    for k in n_gram_dict_test.keys():
        context = k[0]


        if(k in n_gram_dict.keys()): 
            #print("da")
            if(context in n_1_gram_dict.keys()):
                mle_test[k] = (n_gram_dict[k]+1)/(n_1_gram_dict[context]+len(list(set(train))))
                #print("da")
            else:
                (n_gram_dict[k]+1)/(len(train))
                #print("dae")
        else:
            mle_test[k] = 1/len(train)
    return mle_test, mle_train

In [18]:
mle_test_add1, mle_train_add1 = smoothing(words_train,words_test)

In [19]:
mle_test_add1

{('in', 'two'): 1.4025245441795232e-05,
 ('two', 'hours'): 0.0004878048780487805,
 ('hours', 'we'): 1.4025245441795232e-05,
 ('we', 'must'): 0.0020123839009287925,
 ('must', 'be'): 0.0061468780329990294,
 ('be', 'on'): 0.0003056234718826406,
 ('on', 'the'): 0.017998105462582886,
 ('the', 'scene'): 0.00046851574212893553,
 ('scene', 'of'): 0.0008263097008758883,
 ('of', 'then'): 0.0007227174174897615,
 ('then', 'suddenly'): 0.0008098477486232588,
 ('suddenly', 'another'): 1.4025245441795232e-05,
 ('another', 'sound'): 1.4025245441795232e-05,
 ('sound', 'became'): 1.4025245441795232e-05,
 ('became', 'very'): 1.4025245441795232e-05,
 ('very', 'soothing'): 1.4025245441795232e-05,
 ('soothing', 'like'): 1.4025245441795232e-05,
 ('like', 'that'): 0.0008172605426610003,
 ('that', 'of'): 0.0021843003412969283,
 ('of', 'a'): 0.017104312213924354,
 ('a', 'small'): 0.004515499145716377,
 ('small', 'jet'): 1.4025245441795232e-05,
 ('jet', 'of'): 1.4025245441795232e-05,
 ('of', 'steam'): 1.40252454

MLE of those bigrams decreased whose MLW was relativatively high like for ('the','scene') decresed from 0.666 to 0.0004. other example : (upon , the). The reason might be beacause of addition of size of vocabulary  in gthe denominator

Q6. implementing good turing smoothing with bigrams and observing the difference in count

In [20]:
bi_ = Counter(list(ngrams(words_train+words_test, 2)))

In [21]:
N = Counter(list(bi_.values()))

In [22]:
count={}
for i in range(1,11):
    #print(i)
    count[i] = (i+1)*(N[i+1]/N[i])

In [23]:
count

{1: 0.30523227699152133,
 2: 1.1419518377693283,
 3: 2.1420643729189788,
 4: 2.7979274611398965,
 5: 4.266666666666667,
 6: 4.5390625,
 7: 5.975903614457831,
 8: 8.17741935483871,
 9: 7.928994082840237,
 10: 8.208955223880597}

The discounting value is approximately 0.7 in majority of cases

Q7. good_turing(train,test):  test and train are the set of words on which we want to test and train. 
    The function returns two dictionaries each containing MLE values with turing smoothing for each ngram for train and test set

In [24]:
def good_turing(train,test):
    mle_train = {}
    mle_test ={}


    n_gram = list(ngrams(train, 2))
    n_gram_dict = Counter(n_gram)
    N = Counter(list(n_gram_dict.values()))
    n_1_gram = words_train
    n_1_gram_dict = Counter(n_1_gram)
    N_1_ = Counter(list(n_1_gram_dict.values()))
    gt_n_train = {}
    gt_n_1_train = {}

    for i in n_gram_dict.keys():
        c = n_gram_dict[i]
        if(N[c+1] == 0):
            c_eff = c
         
        else:
            c_eff = float(c+1)*(N[c+1]/N[c])
        gt_n_train[i] = c_eff
        
    for i in n_1_gram_dict.keys():
        c = n_1_gram_dict[i]
        if(N_1_[c+1] ==0):
            c_eff = c 
        else:

            c_eff = float(c+1)*(N_1_[c+1]/N_1_[c])

        gt_n_1_train[i] = c_eff
 
    c_n_eff_0 = N[1]/len(n_gram)
    c_n_1_eff_0 = N_1_[1]/len(n_1_gram)

    for k in n_gram_dict.keys():
        context = k[0]
        mle_train[k] = gt_n_train[k]/gt_n_1_train[context]
 
      
    n_gram_test = list(ngrams(test, 2))
    n_gram_dict_test = Counter(n_gram_test)

    
    for k in n_gram_dict_test.keys():
        context = k[0]


        if(k in gt_n_train.keys()): 

            if(context in gt_n_1_train.keys()):

                mle_test[k] = (gt_n_train[k])/(gt_n_1_train[context])

            else:
                (gt_n_train[k])/c_n_1_eff_0

        else:
            mle_test[k] = N[1]/len(n_gram)

    return mle_test, mle_train

In [25]:
mle_test_turing, mle_train_turing = good_turing(words_train,words_test)

In [26]:
mle_test_turing

{('in', 'two'): 0.37628858749772087,
 ('two', 'hours'): 0.01072717895822589,
 ('hours', 'we'): 0.37628858749772087,
 ('we', 'must'): 0.020585197934595525,
 ('must', 'be'): 0.2773722627737226,
 ('be', 'on'): 0.0005793390052683547,
 ('on', 'the'): 0.39100346020761245,
 ('the', 'scene'): 0.0006124879709885331,
 ('scene', 'of'): 0.4170306254797639,
 ('of', 'then'): 0.001782402877799959,
 ('then', 'suddenly'): 0.02196885148654219,
 ('suddenly', 'another'): 0.37628858749772087,
 ('another', 'sound'): 0.37628858749772087,
 ('sound', 'became'): 0.37628858749772087,
 ('became', 'very'): 0.37628858749772087,
 ('very', 'soothing'): 0.37628858749772087,
 ('soothing', 'like'): 0.37628858749772087,
 ('like', 'that'): 0.07659410383145791,
 ('that', 'of'): 0.010326086956521738,
 ('of', 'a'): 0.06247230837394772,
 ('a', 'small'): 0.0172173103769195,
 ('small', 'jet'): 0.37628858749772087,
 ('jet', 'of'): 0.37628858749772087,
 ('of', 'steam'): 0.37628858749772087,
 ('steam', 'escaping'): 0.3762885874977

perplexity(test,model) takes randomly 5 sentences from the test set and outputs the average perplexity of a bigram model

In [40]:
def perplexity(test,model):
    itr = 5
    rand_sen = [test[i] for i in np.random.choice(len(test),itr)]
    #print(len(rand_sen))
    perp_ = 0
    n = 2
    seq_len = 10
    words_ = []
    for i in range(itr):
        words_ = []
        words_ =[word.lower() for word in rand_sen[i].split() if word.isalpha()]
        n_grams =list(ngrams(words_[:seq_len], n))
        prod = 1.
        for j in range(len(n_grams)):
            
            prod *= model[n_grams[j]]
 
        perp_ += 1/(prod**(1./seq_len))
    return (1./itr)*(perp_)


Perplexity for bigram model with turing smoothing

In [43]:
perplexity(X_test,mle_test_turing)


10.27091890635264

Perplexity for bigram model with add 1 smoothing

In [42]:
perplexity(X_test,mle_test_add1)

1156.823014171144